# Lab: Aprendizaje por refuerzo multi-agente (IV)

En los notebooks anteriores hemos visto los siguientes algoritmos:

* Joint Action Learning with Game Theory (JAL-GT), que permite resolver juegos con diferentes conceptos de solución, en dos versiones:
  * Valores Q en forma de matriz $N \times S \times AS$.
  * Valores Q como redes neuronales, con input el estado en un vector _one-hot_ y output un valor para cada acción conjunta.
* Joint Action Learning with Agent Modelling (JAL-AM), modelando las políticas de los demás agentes de manera probabilística, permitiendo calcular las mejores respuestas a las acciones conjuntas.
* Win or Learn Fast, un algoritmo a medio camino entre un método basado en valor (_value-based_) y un método de gradiente de política (_policy-based_), aunque sin realmente calcular un gradiente ya que son fijos como parámetro inicial: tasa de aprendizaje cuando se pierde y tasa de aprendizaje cuando se gana.

En estos dos últimos notebooks vamos a ver dos algoritmos nuevos que son de referencia en aprendizaje por refuerzo de un agente pero que también se utilizan en entornos multiagente, reduciendo el problema MARL a un problema de un agente (como aprendizaje independiente), debido a su adaptabilidad (al usar redes neuronales para representar sus elementos) y a ciertas mejoras que permiten una mayor estabilidad de aprendizaje. Esta mejora en la estabilidad permite resolver, en muchos casos, el problema de la no-estacionariedad. Sin embargo, no ofrecen ninguna garantía de convergencia, por lo que, en un caso general y especialmente en aquellos escenarios que combinen intereses similares y conflictivos a la vez, la mejor opción es la de utilizar algoritmos específicos de MARL (e.g. MAPPO o MADDPG).

# Deep Q Network

El primer algoritmo que veremos es Deep Q Network. Es una evolución de Q-Learning que es, junto con Proximal Policy Optimization (PPO), el algoritmo de aprendizaje por refuerzo más usado a día de hoy.

La idea fundamental de este algoritmo es reemplazar la tabla/vector/matriz de valores Q por una red neuronal, llamada modelo $Q$. El uso de redes neuronales permite la representación y ajuste de este valor en entornos continuos o con un tamaño intratable de estados.

Sin embargo, un problema inherente a los algoritmos _value-based_ es la frecuente falta de estabilidad en el aprendizaje, debido a que la política se basa en los mismos valores que estamos constantemente cambiando, potencialmente causando problemas de no-estacionariedad y convergencia entre los cambios en los valores Q y el comportamiento que esos mismos cambios causan. Este problema aún se agrava más cuando, como en el caso de Q-Learning con redes neuronales, ampliamos el tamaño del espacio de estados (e.g. de discreto a continuo) o el espacio de acciones.

DQN implementa una mejora que pretende solventar este problema: añadir otro modelo de valor $Q'$, de estructura idéntica a la que se está aprendiendo, que es se mantiene estática la mayor parte del tiempo y que se actualiza con los valores de $Q$, con una baja frecuencia. Este modelo $Q'$ se utiliza para calcular el _TD target_ (la diferencia temporal). Al ser un valor casi siempre estático, ayuda a mejorar la convergencia.

Para mejorar la estabilidad del entrenamiento todavía más, DQN se fundamenta en una segunda mejora: el entrenamiento basado en un buffer de experiencias (donde cada experiencia es una tupla $(s, a, r, s')$) que se guardan en una cola de tamaño limitado donde se guardan las experiencias más recientes. Cada iteración, se entrena en base a una muestra aleatoria del contenido de este buffer. De esta manera, se busca evitar cambios sesgados y repentinos a los valores Q, al calcular los gradientes en base a lotes (_batches_) de experiencias en lugar de procesarlas una a una.

El pseudocódigo del algoritmo DQN ([Multi-Agent Reinforcement Learning, Albrecht et al. 2024](https://www.marl-book.com/download/), Sección 9.3.1, p. 225) es:

<div>
<img src="dqn.png" width="500"/>
</div>

DQN, utilizado como algoritmo de aprendizaje independiente, no ofrece garantías de convergencia a ningún concepto de solución. Sin embargo, en muchos entornos, especialmente en aquellos que ofrezcan situaciones de intereses no mixtos, permite aproximar equilibrios de Nash de manera similar a como se consigue con WoLF (ver notebook `10_wolf.ipynb`) debido a los métodos de mejora de estabilidad aplicados.

## Instalación y configuración

### Dependencias

In [ ]:
!pip --quiet install rlcard pettingzoo seaborn matplotlib numpy pandas tinynn pygame scikit-learn tqdm

### Imports necesarios

In [ ]:
import abc
import itertools
import random
import copy
from collections import deque

from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tinynn.core.layer import Dense, ReLU
from tinynn.core.loss import MSE
from tinynn.core.model import Model
from tinynn.core.net import Net
from tinynn.core.optimizer import SGD

### Código de anteriores notebooks

In [ ]:
def pretty_print_array(ar):
    return np.array_str(np.array(ar), precision=2, suppress_small=True)

def draw_history(history, title):
    data = pd.DataFrame({'Episode': range(1, len(history) + 1), title: history})
    plt.figure(figsize=(10, 6))
    sns.lineplot(x='Episode', y=title, data=data)

    plt.title(title + ' Over Episodes')
    plt.xlabel('Episode')
    plt.ylabel(title)
    plt.grid(True)
    plt.tight_layout()

    plt.show()

In [ ]:
class GameModel:
    def __init__(self, num_agents, num_actions):
        self.num_agents = num_agents
        self.num_actions = num_actions
        self.action_space = self.generate_action_space()
        self.action_space_index = {joint_action: idx for idx, joint_action in enumerate(self.action_space)}

    def generate_action_space(self):
        actions_by_players = []
        for agent_id in range(self.num_agents):
            actions_by_players.append(range(self.num_actions))
        all_joint_actions = itertools.product(*actions_by_players)
        return [tuple(l) for l in all_joint_actions]

In [ ]:
class MARLAlgorithm(abc.ABC):
    @abc.abstractmethod
    def learn(self, joint_action, rewards, state, next_state, terminated, truncated):
        pass

    @abc.abstractmethod
    def select_action(self, state):
        pass

## Algoritmo DQN

En la constructora podemos ver los nuevos elementos:

* Buffer de experiencias: `experience_replay`
* Modelo $Q$: `q_value`
* Modelo $Q'$: `q_target`

Además, para el manejo del buffer de experiencias tenemos parámetros nuevos:

* `buffer_size` determina el tamaño del buffer de experiencias. Si se llena, las experiencias más antiguas van desapareciendo en favor de las nuevas.
* `batch_size` es el tamaño de la muestra que se extrae aleatoriamente del buffer para el entreno.
* `update_steps` es la frecuencia de actualización del modelo $Q'$: el número de veces que el modelo $Q$ se ha actualizado.

In [ ]:
class DQN(MARLAlgorithm):
    def __init__(self, agent_id, game: GameModel,
                 buffer_size=4096, batch_size=512, update_steps=1024, # Parámetros nuevos
                 gamma=0.95, alpha=0.5, epsilon=0.2, seed=42):
        self.agent_id = agent_id
        self.game = game
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.rng = random.Random(seed)

        # Elementos nuevos con respecto a Q-Learning
        self.experience_replay = deque(maxlen=buffer_size)
        net_q_value = Net([Dense(64), ReLU(),
                           Dense(32), ReLU(),
                           Dense(self.game.num_actions)])
        net_q_target = Net([Dense(64), ReLU(),
                            Dense(32), ReLU(),
                            Dense(self.game.num_actions)])
        optimizers = [SGD(lr=self.alpha) for _ in range(2)]
        self.q_value = Model(net=net_q_value, loss=MSE(), optimizer=optimizers[0])
        self.q_target = Model(net=net_q_target, loss=MSE(), optimizer=optimizers[1])
        self.update_steps = update_steps
        self.batch_size = batch_size
        self.step_counter = 0
        self.metrics = {"td_error": [], "loss": []}

Definimos un método `update_q` que, dada una muestra de experiencias, calcula las diferencias temporales para cada experiencia y calcula y aplica los gradientes:

In [ ]:
class DQN(DQN):
    def update_q(self, experience_samples):
        # experience_samples = [experience_1, ..., experience_N]
        # experience_n = (s, a, r, s')
        states = np.array([e[0] for e in experience_samples])
        next_states = np.array([e[3] for e in experience_samples])

        predicted_q_values = self.q_value.forward(states)
        target_q_values = copy.deepcopy(predicted_q_values)
        static_max_q_values = np.max(self.q_target.forward(next_states), axis=1)

        td_error = []
        for idx, (s, a, r, next_s, terminated) in enumerate(experience_samples):
            if terminated:
                target_q_values[idx][a] = r
            else:
                target_q_values[idx][a] = r + self.gamma * static_max_q_values[idx]
            td_error = target_q_values[idx][a] - predicted_q_values[idx][a]
        loss, grads = self.q_value.backward(predicted_q_values, target_q_values)
        self.q_value.apply_grads(grads)


        self.step_counter += 1
        if self.step_counter > self.update_steps:
            # Si hemos entrenado self.update_steps veces, actualizamos Q'
            q_value_params = self.q_value.net.params
            q_value_params_copy = copy.deepcopy(q_value_params)
            self.q_target.net.params = q_value_params_copy
            self.step_counter = 0  # Reseteamos el contador de actualizaciones

        return np.average(td_error), loss

El método `learn` registra la experiencia en el buffer y, si hay suficientes elementos para muestrear, entrenamos:

In [ ]:
class DQN(DQN):
    def learn(self, joint_action, rewards, state, next_state, terminated, truncated):
        experience = (state[self.agent_id], joint_action[self.agent_id],
                      rewards[self.agent_id], next_state[self.agent_id],
                      terminated[self.agent_id])
        self.experience_replay.append(experience)
        if len(self.experience_replay) >= self.batch_size:
            experience_samples = self.rng.sample(self.experience_replay, self.batch_size)
            td_error, loss = self.update_q(experience_samples)
            # Guardamos el error de diferencia temporal y la pérdida de la red neuronal para estadísticas posteriores
            self.metrics['td_error'].append(abs(td_error))
            self.metrics['loss'].append(abs(loss))

Finalmente, utilizamos el máximo valor Q para un estado para determinar la política en dicho estado, teniendo en cuenta que si estamos en entrenamiento aplicamos $\epsilon$-greediness:

In [ ]:
class DQN(DQN):
    def select_action(self, state, train=True):
        if train and self.rng.random() < self.epsilon:
            return self.rng.choice(range(self.game.num_actions))
        else:
            return np.argmax(self.q_value.forward(np.array([state]))[0])

## Entrenamiento

En este caso vamos a trabajar con un entorno continuo: el [pursuit_v4](https://pettingzoo.farama.org/environments/sisl/pursuit/). Echad un vistazo a la documentación para entender el objetivo de entrenamiento y cómo se representa el estado y las acciones. Debido a que el estado tiene una representación matricial vamos a aplicar un `flatten` para convertirlo en un vector:

In [ ]:
def obs_to_state(obs):
    return obs.flatten()

Esta es la función de entrenamiento, parecida a la utilizada en notebooks anteriores. En este caso hemos introducido un cambio, que consiste en permitir definir una serie de escenarios aleatorios. Cada epoch consiste en hacer un entrenamiento completo en cada uno de los escenarios. Por lo tanto, si definimos 5 escenarios y 10 epochs, haremos un total de 50 entrenamientos:

In [ ]:
def train_agents(env, scenarios, epochs, gammas, alphas, epsilons, seeds):
    env.reset()
    agent_strings = env.agents
    num_agents = len(env.agents)

    # Crear modelo de juego y algoritmos para cada agente
    game_model = [GameModel(num_agents=num_agents, num_actions=env.action_space(agent_strings[i]).n)
                  for i in range(num_agents)]
    algorithms = [DQN(agent_id, game_model[agent_id], buffer_size=50000, batch_size=128, update_steps=10000,
                      gamma=gammas[agent_id], alpha=alphas[agent_id], epsilon=epsilons[agent_id],
                      seed=seeds[agent_id])
                  for agent_id in range(num_agents)]

    # Métricas a guardar: recompensas y acciones
    cumulative_rewards = [[0] * num_agents]
    actions_played = [[]] * num_agents

    for _ in tqdm(range(epochs)): # Recorridos completos a todos los escenarios
        for idx in range(scenarios): # Escenarios aleatorios
            # Observación inicial
            observations, infos = env.reset(seed=idx)
            states = [obs_to_state(observations[agent_strings[i]]) for i in range(num_agents)]
            episode_rewards = [0] * num_agents
            while env.agents:
                env.render()

                # Selección de acciones
                actions = []
                petting_zoo_actions = {}
                for i in range(num_agents):
                    actions.append(algorithms[i].select_action(states[i]))
                    actions_played[i].append(actions[i])
                    petting_zoo_actions[agent_strings[i]] = actions[i]

                # Actualización de entorno
                observations, rewards, terminations, truncations, infos = env.step(petting_zoo_actions)

                # Conversión de formato petting zoo -> lista
                next_states = []
                indexed_rewards = []
                indexed_terminations = []
                indexed_truncations = []
                for i in range(num_agents):
                    next_states.append(obs_to_state(observations[agent_strings[i]]))
                    indexed_rewards.append(rewards[agent_strings[i]])
                    indexed_terminations.append(terminations[agent_strings[i]])
                    indexed_truncations.append(truncations[agent_strings[i]])
                    episode_rewards[i] += indexed_rewards[i]

                # Entrenamiento
                for i in range(num_agents):
                    algorithms[i].learn(actions, indexed_rewards, states, next_states,
                                        indexed_terminations, indexed_truncations)
                states = next_states
            cumulative_rewards.append([cumulative_rewards[-1][i] + indexed_rewards[i]
                                       for i in range(num_agents)])
            env.close()
    return game_model, algorithms, cumulative_rewards, actions_played

### Configuración y ejecución del entrenamiento

En este caso comenzamos con algunos valores pequeños para muchos parámetros, menores que los que se muestran en la documentación.

**Antes de entrenar, intentad predecir el efecto en el entrenamiento que tendrá cada uno de los parámetros.** Pensad que la complejidad del entrenamiento es mucho mayor que la que hemos podido ver en anteriores notebooks, ya sea en entrenamiento de un agente o multiagente. Los entrenamientos pueden ser lentos, por lo que una exploración a ciegas puede ser bastante ineficiente.

Si queréis ver cómo se comportan los agentes, poned `render_mode="human"`, aunque puede que vaya algo más lento.

In [ ]:
from pettingzoo.sisl import pursuit_v4

env = pursuit_v4.parallel_env(render_mode="rgb_array", x_size=8, y_size=8,
                              n_evaders=1, n_pursuers=2, surround=False,
                              obs_range=2, max_cycles=50)
env.reset()
num_agents = len(env.agents)
game_model, algorithms, cumulative_rewards, actions_played = \
    train_agents(env, scenarios=10, epochs=10, gammas=[0.95] * num_agents,
                 alphas=[0.1] * num_agents, epsilons=[0.2] * num_agents,
                 seeds=[i for i in range(num_agents)])

print(f"Recompensas acumuladas: {cumulative_rewards[-1][0]}, {cumulative_rewards[-1][1]}")

In [ ]:
draw_history([cumulative_rewards[idx][0] for idx in range(len(cumulative_rewards))], "Rewards")
draw_history(algorithms[0].metrics['td_error'], "TD Error")
draw_history(algorithms[0].metrics['loss'], "Loss")

### Evaluación

Con este código podéis probar escenarios aleatorios. **Comprobad si los algoritmos que habéis entrenado han generalizado lo suficiente como para poder resolver escenarios nuevos.**

In [ ]:
env = pursuit_v4.parallel_env(render_mode="human", x_size=8, y_size=8,
                              n_evaders=1, n_pursuers=2, surround=False,
                              obs_range=2, max_cycles=1000)
observations, infos = env.reset(seed=random.randint(0, 10000))
agent_strings = env.agents
all_rewards = [0]

states = [obs_to_state(observations[agent_strings[i]]) for i in range(num_agents)]
while env.agents:
    env.render()
    actions = [algorithms[i].select_action(states[i]) for i in range(num_agents)]
    petting_zoo_actions = {agent_strings[i]: actions[i] for i in range(num_agents)}
    observations, rewards, terminations, truncations, infos = env.step(petting_zoo_actions)
    next_states = [obs_to_state(observations[agent_strings[i]]) for i in range(num_agents)]
    indexed_rewards = [rewards[agent_strings[i]] for i in range(num_agents)]
    indexed_terminations = [terminations[agent_strings[i]] for i in range(num_agents)]
    all_rewards.append(all_rewards[-1] + sum(indexed_rewards))
    states = next_states
if any(indexed_terminations):
    print("Todos los evaders han sido cazados!")
env.close()
draw_history(all_rewards, "Cumulative reward")